In [34]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.documents import Document
from langchain_chroma import Chroma

import uuid
from langchain_community.document_loaders import PyPDFLoader

In [35]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=510, 
    chunk_overlap=50,
    length_function=len,
    separators=["\n", ".", "\n\n"]
)

In [36]:
# Probar distintos modelos de embeddings
embeddings_generator = OllamaEmbeddings(
    model='mxbai-embed-large:latest',
)

In [37]:
chroma_db_path = "./database/chroma_db"

vector_store = Chroma(
    persist_directory=chroma_db_path,
    embedding_function=embeddings_generator,
    collection_name="kanji"
)

In [46]:
# Documentos txt
txt_path = './database/docs/txt/kanji-3.txt'
txt_text = ''

with open(txt_path, 'r', encoding='utf-8') as file:
    txt_text = file.read()

print(len(txt_text))

10479


In [47]:
text_chunks = text_splitter.split_text(txt_text)
print(len(text_chunks))

29


In [48]:
for chunk in text_chunks:
    document = Document(
        id=str(uuid.uuid4()),
        page_content=chunk,
        metadata={
            'name': 'kanji',
            'source': 'https://www.bokksu.com/blogs/news/what-is-kanji?srsltid=AfmBOopsm5qJHRqfC6NRbI2HAvDPODxTtW1xp3c4DcPnmJKXveDpAL3i',
            'generation': 6,
            'image_src': ''
        }
    )

    vector_store.add_documents([document])

In [49]:
vector_store.similarity_search("Que es un kanji?", k=5)

[Document(id='db73f200-2722-4d69-8110-a97ecd0fb5f9', metadata={'image_src': 'https://mirandohaciajapon.com/wp-content/uploads/2016/09/kanji-japon-768x574.jpg', 'name': 'kanji', 'generation': 6, 'source': 'https://mirandohaciajapon.com/el-kanji-su-utilidad-y-funcion/'}, page_content='Primero que todo ¿Qué es el Kanji?\n\nPara empezar, el kanji es uno de los tres sistemas de escritura que se usan para escribir en japonés. Este sistema tiene su origen en China, y fue llevado a Japón hace cientos de años, cuando el japonés era solo un idioma hablado. Seguramente los habrás visto en muchos lugares.\n\nSon esas letras que tienen muchas «rayitas» y parecen dibujos. Esos son los famosos kanjis, y en el japonés se usan principalmente para expresar palabras y conceptos.'),
 Document(id='ed407d8f-1326-4eb2-b908-5790181883ca', metadata={'generation': 6, 'image_src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Kanji_furigana.svg/163px-Kanji_furigana.svg.png', 'source': 'https://es.wi

In [57]:
pdf_document = PyPDFLoader('./database/docs/PDF/KanjiParaRecordar.pdf')
pdf_pages = pdf_document.load()

In [58]:
pdf_chunks = text_splitter.split_documents(pdf_pages)
print(len(pdf_chunks))

1454


In [59]:
for chunk in pdf_chunks:
    chunk.id = str(uuid.uuid4())
    chunk.metadata = {
        "name": "definicion"
    }

In [60]:
vector_store.add_documents(pdf_chunks)

['d57bdab1-986f-4547-a003-2b003ee12b92',
 'bdb78a5f-ae60-4cb3-b85b-3c3d80b87578',
 '21e4405a-6261-4423-a333-999c9d58eba0',
 '6100f791-83a6-4dc0-a725-df7ec6b6bcda',
 '93703788-0e42-43eb-ad46-4bc917c7c223',
 '757de8e2-62c6-4177-9ff6-b5983c76a62d',
 '96d614a9-7192-4563-bc51-2c4b5b83e7ce',
 '600d9173-db88-4eee-b66e-65ee537dc3a3',
 '4e06a7ec-c88a-4880-94a0-5fa6cc4b8a7f',
 '56f51240-4af3-46fb-8d87-e1fde9fe22aa',
 '6254970b-8581-4b1b-93be-c6b25a669ef8',
 '83c9a7ff-5078-497d-9e7e-2c0d1c8dfa48',
 '82a0d09b-2e67-404a-8627-b370a39dd66c',
 '99ab7238-ee3d-4207-ac40-da757b01dbf3',
 'af757c26-8e4e-40e9-a83a-c5ece4ab465b',
 '4efd281c-f4d5-4856-bd4b-5e0a0d82d876',
 '289b0ccc-bd85-44c8-aef7-c459da685f8b',
 'a4957156-018e-4918-9891-1aeadcc376e8',
 'b1f45dc4-1d14-4280-95ac-224f43bbf0b6',
 '5b284837-2ed8-4194-bf8b-05b4cd81950e',
 '2c726fbb-91a2-4189-a233-23886a1b6ee1',
 '273c7191-ded9-4dfc-bd77-0f235bb3b713',
 '5e43947a-a352-464a-baf8-c588eaaa0f44',
 '7492f497-dc41-45c1-93bf-b3382d4bbcdf',
 '7df1f54f-32eb-

In [61]:
vector_store.similarity_search("Que es un kanji?", k=5)

[Document(id='b69efe79-777d-4a6e-b369-4a5cff846dcf', metadata={'name': 'definicion'}, page_content='responder a la pregunta: "¿Qué es el kanji?".\n forma parte de la escritura japonesa, por lo que el japonés y  están inextricablemente unidos. En la\nmayoría de las oraciones en japonés,  se combina con uno o ambos de los otros sistemas de escritura\njaponeses:  y  . Así que, si preguntas "¿Qué es  ?", no te sorprendas si la respuesta es larga.\nLa di�cultad para aprender  depende del enfoque y la dedicación. Aprender los 2136  de uso común en un'),
 Document(id='3c20021c-b10b-4f0d-b2e5-4110a05f8f3b', metadata={'name': 'definicion'}, page_content='▪ ¿Qué es kanji?\n▪ ¿Cómo se leen los kanji?\n▪ Uso de la puntuación japonesa con kanji\xa0\n▪ Kanjis japoneses importantes que debes saber\xa0\n▪ Preguntas frecuentes sobre los kanji japoneses'),
 Document(id='db73f200-2722-4d69-8110-a97ecd0fb5f9', metadata={'image_src': 'https://mirandohaciajapon.com/wp-content/uploads/2016/09/kanji-japon-768

In [55]:

# Para eliminar la colección
#vector_store.delete_collection()
#vector_store.reset_collection()